To copy this template: File -> Save a Copy in Drive

***DISCLAIMER**: In case of any discrepancy in the assignment instruction, please refer to the `PDF` document.*

# Problem 2 - Neural Network Training and Backpropagation

## 2.1

In [ ]:
# sigmoid()
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

## 2.2

In [ ]:
# forward_propagate()

## 2.3

In [ ]:
# cost() without regularization

In [ ]:
# cost() with regularization

## 2.4

In [ ]:
# sigmoid_gradient()

## 2.5

In [ ]:
# backprop() without regularization

In [ ]:
# backprop() with regularization

## 2.6

In [ ]:
# your code starts here

## 2.7

In [ ]:
# your code starts here

## 2.8

**Answer:**
